### Projeto de conclusão do módulo III - Técnicas de Programação
#### Elaborado por Maína Alexandre

In [ ]:
import pandas as pd
from geopy.distance import geodesic
from unidecode import unidecode
import re
import random
import matplotlib.pyplot as plt

In [ ]:
# Carregando os arquivos .csv para dataframes
df_escolas = pd.read_csv('escolas.csv')
df_material = pd.read_csv('material_didatico.csv')
df_subprefeituras = pd.read_csv('subprefeituras.csv')

In [ ]:
df_escolas.head()

In [ ]:
df_material.head()

In [ ]:
df_subprefeituras.head()

In [ ]:
# renomeando a coluna no df de subprefeituras para usar como chave comum no merge
df_subprefeituras.rename(columns={'nome': 'BAIRRO'}, inplace=True)

# tratando o nome dos bairros nos dois dataframes
df_subprefeituras['BAIRRO'] = df_subprefeituras['BAIRRO'].str.upper()
df_escolas['BAIRRO'] = df_escolas['BAIRRO'].str.upper()

In [ ]:
# unindo escolas com subprefeituras
df_junto = df_escolas.merge(df_subprefeituras, on='BAIRRO', how='left')
df_junto.head()

In [ ]:
# unindo com materiais
df_entregas = df_junto.merge(df_material, on='id', how='left')
df_entregas.head()

In [ ]:
df_entregas.columns

In [ ]:
df_entregas['Escolas_Postos'] = df_entregas['Escolas_Postos'].str.upper()
df_entregas['ENDEREÇO '] = df_entregas['ENDEREÇO '].str.upper()
df_entregas['subprefeitura'] = df_entregas['subprefeitura'].str.upper()


In [ ]:
df_entregas.shape

In [ ]:
df_entregas.isnull().sum()

In [ ]:
df_entregas[df_entregas.isnull().any(axis=1)]

In [ ]:
df_entregas['Quantidade'].unique()

In [ ]:
df_entregas['Quantidade'] =  df_entregas['Quantidade'].apply(lambda x: x.strip("' ").strip() if isinstance(x, str) else x)
df_entregas['Quantidade'] =  df_entregas['Quantidade'].fillna(0)
df_entregas['Quantidade'] =  df_entregas['Quantidade'].replace({'NaN':0, '':0})
df_entregas['Quantidade'] =  df_entregas['Quantidade'].astype(int)

df_entregas['Quantidade'].unique()



In [ ]:
print('Média:', df_entregas['Quantidade'].mean())
print('Mediana:', df_entregas['Quantidade'].median())


In [ ]:
df_entregas['Quantidade'] =  df_entregas['Quantidade'].replace(0, 311)
df_entregas['Quantidade'].unique()

In [ ]:
df_entregas['BAIRRO'].unique()

In [ ]:
pd.set_option('display.float_format', '{:.5f}'.format)
df_entregas['lat'] =  df_entregas['lat'].str.replace(',', '.').astype(float).round(5)
df_entregas['lat'].unique()

In [ ]:
df_entregas['lon'] =  df_entregas['lon'].str.replace(',', '.').astype(float).round(5)
df_entregas['lon'].unique()

In [ ]:
df_entregas[(df_entregas['lat'] > 90) | (df_entregas['lat'] < -90)]

In [ ]:
df_entregas['lat'] = df_entregas['lat'].apply(lambda x: x / 1000000 if x < -90 else x)

In [ ]:
df_entregas[(df_entregas['lon'] > 90) | (df_entregas['lon'] < -90)]

In [ ]:
df_entregas['lon'] = df_entregas['lon'].apply(lambda x: x / 1000000 if x < -90 else x)

In [ ]:
df_entregas.describe()

In [ ]:
def encontrar_subprefeitura_proxima(bairro):
    if pd.notna(bairro['subprefeitura']):
        return bairro['subprefeitura']

    coordenadas_bairro = (bairro['lat'], bairro['lon'])
    bairros_com_subprefeitura = df_entregas.dropna(subset=['subprefeitura'])
    distancias = bairros_com_subprefeitura.apply(
        lambda x: geodesic(coordenadas_bairro, (x['lat'], x['lon'])).kilometers,
        axis=1
    )
    bairro_mais_proximo = bairros_com_subprefeitura.loc[distancias.idxmin()]
    return bairro_mais_proximo['subprefeitura']

df_entregas['subprefeitura'] = df_entregas.apply(encontrar_subprefeitura_proxima, axis=1)

df_entregas

In [ ]:
df_entregas['subprefeitura'].unique()

In [ ]:
novas_colunas = {
    'Escolas_Postos': 'escola_posto',
    'BAIRRO': 'bairro',
    'ENDEREÇO ': 'endereco',
    'lat': 'latitude',
    'lon': 'longitude',
    'Quantidade': 'quantidade'
}
df_entregas = df_entregas.rename(columns=novas_colunas)
df_entregas.head()

In [ ]:
df_entregas['id'].dtype

In [ ]:
df_entregas['id'] = df_entregas['id'].astype(str).str.zfill(3)
df_entregas['id'].describe()

In [ ]:
df_entregas = df_entregas.drop_duplicates(subset='id', keep='first')

In [ ]:
df_entregas['id'].describe()

In [ ]:
df_entregas['escola_posto'] = df_entregas['escola_posto'].apply(lambda x: unidecode(x))
df_entregas['bairro'] = df_entregas['bairro'].apply(lambda x: unidecode(x))
df_entregas['endereco'] = df_entregas['endereco'].apply(lambda x: unidecode(x))
df_entregas['subprefeitura'] = df_entregas['subprefeitura'].apply(lambda x: unidecode(x))

df_entregas


In [ ]:
df_entregas[df_entregas['endereco'].str.contains(r'\.')]

In [ ]:
df_entregas['endereco'] = df_entregas['endereco'].replace('R\.', 'RUA', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('AV\.', 'AVENIDA', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('AV ', 'AVENIDA', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('PCA\.', 'PRACA', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('ESTR\.', 'ESTRADA', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('ALM\.', 'ALMIRANTE', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('SEN\.', 'SENADOR', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('MAL\.', 'MARECHAL', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('ENG\.', 'ENGENHEIRO', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('SD\.', 'SOLDADO', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('CONJ\.', 'CONJUNTO', regex=True)


In [ ]:
df_entregas[df_entregas['endereco'].str.contains(r'\.')]

In [ ]:
df_entregas[df_entregas['endereco'].str.contains(r'\/')]

In [ ]:
df_entregas['endereco'] = df_entregas['endereco'].replace('\. S\/No', 'S/N', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('S\/N\.o', 'S/N', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('S\/N\.deg', 'S/N', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('S\/No', 'S/N', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('S\/Ndeg', 'S/N', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('CASTELOS\/N', 'CASTELO S/N', regex=True)


df_entregas[df_entregas['endereco'].str.contains(r'\/')]


In [ ]:
df_entregas['escola_posto'].str[:3].unique()

In [ ]:
df_entregas['escola_posto'] = df_entregas['escola_posto'].replace('CENTRO INTEGRADO DE EDUCACAO PUBLICA', 'CIEP', regex=True)
df_entregas['escola_posto'] = df_entregas['escola_posto'].replace('E\.M.', 'EM ', regex=True)
df_entregas['escola_posto'] = df_entregas['escola_posto'].replace('ESCOLA MUNICIPAL', 'EM', regex=True)
df_entregas['escola_posto'] = df_entregas['escola_posto'].replace('E\.M', 'EM ', regex=True)
df_entregas['escola_posto'] = df_entregas['escola_posto'].replace('EM\.', 'EM ', regex=True)


In [ ]:
df_entregas['escola_posto'].str[:3].unique()

In [ ]:
df_entregas['tipo'] = df_entregas['escola_posto'].str.split().str.get(0)

In [ ]:
df_entregas['tipo'].unique()

In [ ]:
df_entregas

In [ ]:
def extrair_ultima_palavra(endereco):
    palavras = endereco.split()
    ultima_palavras = palavras[-1]
    
    if re.match(r'^\d+$', ultima_palavra) or ultima_palavra == 'S/N':
        return ultima_palavra
    else:
        return None

def atualizar_endereco(endereco):
    palavras = endereco.split()
    ultima_palavra = palavras[-1]
    novo_endereco = ' '.join(palavras[:-1])
    
    if re.match(r'^\d+$', ultima_palavra) or ultima_palavra == 'S/N':
        return str(novo_endereco)
    else:
        return endereco

df_entregas['endereco'] = df_entregas['endereco'].str.replace('[.,;]', '', regex=True)
df_entregas['numero'] = df_entregas['endereco'].apply(extrair_ultimo_palavra)
df_entregas['endereco'] = df_entregas['endereco'].apply(atualizar_endereco)

In [ ]:
df_entregas

In [ ]:
df_entregas[df_entregas['numero'].isna()]

In [ ]:
df_entregas['endereco'] = df_entregas['endereco'].replace(' S\/N ', ' ', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('AVENIDA BRASIL 4040 RUA DA VITORIA', 'AVENIDA BRASIL', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace(' 483\-639', '', regex=True)

df_entregas[df_entregas['numero'].isna()]


In [ ]:
df_entregas.loc[df_entregas['id'] == '520', 'numero'] = '483-639'
df_entregas.loc[df_entregas['id'] == '260', 'numero'] = '4040'



In [ ]:
df_entregas.loc[df_entregas['id'] == '260']

In [ ]:
df_entregas[df_entregas['numero'].isna()]

In [ ]:
df_entregas['numero'].fillna('S/N', inplace=True)

In [ ]:
df_entregas[df_entregas['numero'].isna()]

In [ ]:
df_entregas.rename(columns={'endereco': 'logradouro'}, inplace=True)

In [ ]:
df_entregas

In [ ]:
df_entregas.columns

In [ ]:
nova_ordem = ['id', 'escola_posto', 'tipo', 'logradouro', 'numero', 'bairro', 'subprefeitura', 'quantidade', 'latitude', 'longitude']
df_entregas = df_entregas[nova_ordem]
df_entregas

In [ ]:
df_entregas['subprefeitura'].unique()

In [ ]:
df_qtd_por_subprefeitura = df_entregas.groupby('subprefeitura')['quantidade'].sum().reset_index()
df_qtd_por_subprefeitura.rename(columns={'quantidade': 'quantidade_total'}, inplace=True)
df_qtd_por_subprefeitura

In [ ]:
df_qtd_por_subprefeitura.to_csv("quantidade_por_subprefeitura.csv", index=False)

In [ ]:
import numpy as np
from scipy.spatial import distance_matrix
from sklearn.neighbors import NearestNeighbors

coords = df_entregas[['latitude', 'longitude']].values
dist_matrix = distance_matrix(coords, coords)

n_entregas = len(df_entregas)
nn = NearestNeighbors(n_neighbors=n_entregas - 1, metric='precomputed')
nn.fit(dist_matrix)
route_order = nn.kneighbors(return_distance=False)

rota = [0]
entregas_visitadas = set([0])

for _ in range(n_entregas - 1):
    ultimo_entrega = rota[-1]
    proxima_entrega = None
    menor_distancia = float('inf')
    
    for entrega in route_order[ultimo_entrega]:
        if entrega not in entregas_visitadas:
            if dist_matrix[ultimo_entrega][entrega] < menor_distancia:
                menor_distancia = dist_matrix[ultimo_entrega][entrega]
                proxima_entrega = entrega
                
    if proxima_entrega is not None:
        rota.append(proxima_entrega)
        entregas_visitadas.add(proxima_entrega)

rota_otimizada = df_entregas.iloc[rota]
rota_otimizada

In [ ]:
len(rota_otimizada['id'].unique())

In [ ]:
rota_otimizada.to_csv("rota_otimizada.csv", index=False)

In [ ]:
coords = rota_otimizada[['latitude', 'longitude']].values

plt.figure(figsize=(10, 6))
plt.scatter(coords[:, 1], coords[:, 0], c='blue', marker='o', label='Entregas')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Geolocalizações das Entregas')

plt.plot(coords[:, 1], coords[:, 0], linestyle='-', c='red', label='Rota Otimizada')
plt.scatter(coords[0, 1], coords[0, 0], c='green', marker='s', label='Depósito')

for i, txt in enumerate(rota_otimizada['id']):
    plt.annotate(txt, (coords[i, 1], coords[i, 0]), fontsize=8, ha='right')

plt.legend()
plt.show()